# 비즈덤 활성화하기

1. 아나콘다 프롬프트를 키고 activate 가상환경(pytorch) 를 실행한다.
2. python -m visdom.server 입력해서 서버를 킨다.
3. 크롬에서 http://localhost:8097/ 를 주소창에 킨다.
4. import visdom / vis = visdom.Visdom() 을 입력한다.

In [25]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


In [26]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets

# 실습

In [27]:
# 텍스트를 치니까, 텍스트가 나와용~ 너무조아
vis.text("Hello,",env="main")  
# 이 env = 'main' 으로 설정하면 나중에 한번에 꺼줄 수 있다. 
# 있으나 마나 하긴 함....

'window_38bd3c18391f80'

In [33]:
# 랜덤으로 3 channel 의 200,200 개의 점이 나온다.
# control 누르고 떙기기 가능. 조절이 가능하다.
a = torch.randn(3,200,200) # 
vis.image(a)

'window_38bd3c67f08712'

In [37]:
# 한번에 보여주기
vis.images(torch.Tensor(3,3,28,28)) # 앞에 3 붙이면 한번에 3개 붙여준다.

'window_38bd3ca0e00f76'

# Minist / cifar 데이터셋

In [35]:
MNIST = dsets.MNIST(root="C:/Users/goran/Desktop/Python/Dataset/MINIST",train=True,transform=torchvision.transforms.ToTensor(),download=True)
cifar10 = dsets.CIFAR10(root="C:/Users/goran/Desktop/Python/Dataset/cifar10",train=True,transform=torchvision.transforms.ToTensor(),download=True)

Files already downloaded and verified


In [38]:
data = cifar10.__getitem__(0) # cifar 에서 1번째 데이터를 보고싶어 ~ 
print(data[0].shape) # shape 확인. 3 chennel(컬러) 의 32*32 사진
vis.images(data[0],env="main") 

torch.Size([3, 32, 32])


'window_38bd3cb59c9068'

In [12]:
data = MNIST.__getitem__(0) # Minist 에서 getitem 을 통해서 1번쨰 데이터를 보고싶어
print(data[0].shape) # 0 의 shape 는 어떨까? 
vis.images(data[0],env="main") 

torch.Size([1, 28, 28])


'window_38bd27bf511b4c'

# 데이터 로더를 활용해 한번에 보기

In [39]:
data_loader = torch.utils.data.DataLoader(dataset=MNIST,
                                         batch_size=32,  # batch size 를 설정하자! 
                                         shuffle=False)

In [40]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [41]:
vis.close(env="main") # env 가 main 인 얘들의 창을 다 꺼줭~

''

# Line plot 해보기

In [42]:
# x 값을 넣지 않으면 x축은 0~1 값이 된다.
y_data = torch.randn(5) # 5개의 데이터 형성
plt = vis.line(Y=y_data) # y 데이터로 그릴래요~

In [43]:
# x 값을 넣어주면 x 축의 값이 생긴다.!
x_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=y_data,X=x_data)

In [44]:
y_append = torch.rand(1)
x_append = torch.Tensor([6])
# 추가시킬것들을 넣어주고. 윈도우 이름을 넣어주자. (plt 를 선언했기 때문에, win = plt)
vis.line(Y=y_append,X=x_append,win=plt,update = 'append')

'window_38bd3cfb14bb26'

In [45]:
# 단일 창에서 2개의 lone 그리기
num = torch.Tensor(list(range(0,10))) # num 은 0 ~ 9 의 10개 값(tensor)
print(num.shape)  # shape 는 10 
num = num.view(-1,1) # 
print(num.shape)
num = torch.cat((num,num),dim=1) # torch 를 붙여가지구 y 와 똑같은 모양이 되도록!
print(num.shape) 

plt = vis.line(Y=torch.rand(10,2),X=num) 
# line 2개를 그릴려면, y 와 똑같은 shape 의 X 가 필요하다. 그래서 [10,2] 의 x 값을 가지게 새로 정의

torch.Size([10])
torch.Size([10, 1])
torch.Size([10, 2])


In [46]:
plt = vis.line(Y=y_data,X=x_data,
               opts=dict(title='Test', # 제목 정하기
                         showlegend=True)) # legend 보여주는 설정

In [47]:
plt = vis.line(Y=y_data,X=x_data,opts = dict(title='Test',
                                             legend=['1번'], # 축에 이름넣기
                                             showlegend=True))

In [48]:
plt = vis.line(Y=torch.rand(10,2),X=num,opts=dict(title='Test',
                                                  legend=['1번','2번'], # 2개!
                                                  showlegend=True))

# loss update

In [49]:
# loss update 확인하기
def loss_tracker(loss_plot, loss_value, num):
    # loss_plot = 업데이트할 플롯의 이름
    # loss_value = loss 값이 무엇인지
    # num = loss 값의 x index 
    '''num, loss_value, are Tensor!!'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot, 
             update='append' # append 로 update 해주세요.
             )

In [50]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500): 
    loss = torch.randn(1) + i # 그냥 임의로 집어너은거
    loss_tracker(plt, loss, torch.Tensor([i])) 
    # 사전에 업데이트 되는 loss 값을 사용
    # index 는 그냥 i 사용 (tensor torch 를 사용해야 하므로 tensor 로 감쌈)


In [51]:
vis.close(env="main")

''